In [1]:
from keras import layers, models, optimizers, callbacks  # or tensorflow.keras as keras
# from rpv import load_file
import os
import h5py


Using TensorFlow backend.


In [2]:
def load_file(filename, n_samples):
    with h5py.File(filename) as f:
        data_group = f['all_events']
        data = data_group['hist'][:n_samples][:,:,:,None]
        labels = data_group['y'][:n_samples]
        weights = data_group['weight'][:n_samples]
    return data, labels, weights

In [3]:
# input_dir = '/global/cscratch1/sd/wbhimji/atlas-rpv-images'
input_dir='/global/homes/v/vpa/project/Ice_cube/temp_data/'
train_input, train_labels, train_weights = load_file(os.path.join(input_dir, 'train.h5'), 1000)
valid_input, valid_labels, valid_weights = load_file(os.path.join(input_dir, 'val.h5'), 1000)

In [4]:
print(train_input.shape, valid_input.shape)
print(train_input.shape[1:])
print(train_labels.shape,train_weights.shape)

(1000, 64, 64, 1) (1000, 64, 64, 1)
(64, 64, 1)
(1000,) (1000,)


In [5]:
inputs = layers.Input(shape=train_input.shape[1:])
h = inputs

# Convolutional layers
conv_sizes=[64, 128, 256]
conv_args = dict(kernel_size=(3, 3), activation='relu', padding='same')
for conv_size in conv_sizes:
    h = layers.Conv2D(conv_size, **conv_args)(h)
    h = layers.MaxPooling2D(pool_size=(2, 2))(h)
#h = layers.Dropout(0.5)(h)
h = layers.Flatten()(h)

# Fully connected  layers
h = layers.Dense(64, activation='relu')(h)
#    h = layers.Dropout(0.5)(h)

# Ouptut layer
outputs = layers.Dense(1, activation='sigmoid')(h)

In [6]:
model = models.Model(inputs, outputs)
model.compile(optimizer=optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(train_input, train_labels,
                      batch_size=128,
                      epochs=3,
                      verbose=1,
                      callbacks = [
                        callbacks.ModelCheckpoint('./rpv_weights.h5')
                      ],                      
                )

Epoch 1/3
1000/1000 [==============================] - 10s 10ms/step - loss: 0.6734 - acc: 0.5430
Epoch 2/3
1000/1000 [==============================] - 6s 6ms/step - loss: 0.5149 - acc: 0.5990
Epoch 3/3
1000/1000 [==============================] - 6s 6ms/step - loss: 0.4035 - acc: 0.8490


In [8]:
? model.fit

Signature:  model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, **kwargs)
Docstring:
Trains the model for a given number of epochs (iterations on a dataset).

# Arguments
    x: Numpy array of training data (if the model has a single input),
        or list of Numpy arrays (if the model has multiple inputs).
        If input layers in the model are named, you can also pass a
        dictionary mapping input names to Numpy arrays.
        `x` can be `None` (default) if feeding from
        framework-native tensors (e.g. TensorFlow data tensors).
    y: Numpy array of target (label) data
        (if the model has a single output),
        or list of Numpy arrays (if the model has multiple outputs).
        If output layers in the model are named, you can also pass a
        dictionary mapping output name

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 256)         0         
__________

In [10]:
score = model.evaluate(valid_input,  valid_labels, sample_weight=valid_weights); print ("Validation Accuracy: " + str(score[1] ))

1000/1000 [==============================] - 3s 3ms/step
Validation Accuracy: 0.901
